In [31]:
import numpy as np
import random 
from scipy.stats import unitary_group
from scipy.sparse import csr_matrix, kron
import matplotlib.pyplot as plt
from IPython.display import display
from PIL import Image

In [32]:
# Defining Pauli matrices & CNOTs
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

CNOT = [[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]] # control tensor switch
CNOT_12_34 = kron(CNOT, CNOT)

data_13_24 = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
row_13_24 = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
column_13_24 = np.array([0,1,2,3,5,4,7,6,10,11,8,9,15,14,13,12])
CNOT_13_24 = csr_matrix((data_13_24,(row_13_24,column_13_24)), shape =(16,16))

data_14_23 = np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])
row_14_23 = np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
column_14_23 = np.array([0,1,2,3,6,7,4,5,9,8,11,10,15,14,13,12])
CNOT_14_23 = csr_matrix((data_14_23,(row_14_23,column_14_23)), shape =(16,16))

CNOTS = [CNOT_12_34,CNOT_13_24,CNOT_14_23]

In [33]:
#FUNCTIONS
# defining rotation gates
def Ry(A):
    sn, cs = np.sin(A/2), np.cos(A/2)
    M = np.array([[cs,-sn],[sn, cs]])
    return M
def Rz(A):
    phi = np.exp(-1j*A/2)
    M = np.array([[phi,0],[0,np.conj(phi)]])
    return M
    
#Returns a length-2 array instead of 2x2 matrix. Faster, since it's diagonal. Good for Z rotations!
def Rz2(A):
    phi = np.exp(-1j*A/2)
    return np.array([phi, np.conj(phi)])
    
def Rz2t(A):
    phi = np.exp(-1j*A/2)
    return np.array([[phi], [np.conj(phi)]])

Z2 = np.array([1,-1])
Z2t = np.array([[1],[-1]])
    
# Parameterising the general unitary with rotation gates
def U(A0,A1,A2): # Arguments: Euler Angles
    M = Rz2t(A1) * Ry(A0) * Rz2(A2)
    return M

# derivatives wrt A0,A1,A2
def DA0(A0,A1,A2):
    dA0 = (-1j/2)*(Rz(A1)@Y@Ry(A0)@Rz(A2))
    return dA0
def DA1(A0,A1,A2):
    dA1 = (-1j/2)*(Z@Rz(A1)@Ry(A0)@Rz(A2))
    return dA1
def DA2 (A0,A1,A2):
    dA2 = (-1j/2)*(Rz(A1)@Ry(A0)@Z@Rz(A2))
    return dA2

#Compute U and all three derivatives together, to save computation
def DA_all(A0,A1,A2):
    R1 = Rz2t(A1)
    R0 = Ry(A0)
    R2 = Rz2(A2)
    R02 = R0 * R2
    R102 = R1 * R02
    U = R102
    dA0 = (-0.5j) * R1 * (Y @ R02)
    dA1 = (-0.5j) * Z2t * R102
    dA2 = (-0.5j) * R102 * Z2
    return (U, dA0, dA1, dA2)

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse
    
def parameters(layers): # layers = number of unitary layers, so no-of CNOTs = layers - 1
    A = np.zeros((4*layers,3))
    for i in range(4*layers):
        for j in range(3):
            A[i][j] = random.uniform(0.0, 4*np.pi) 
    return A

#Get the unitary created by these four single-qubit unitaries at layer i.
def U_4squ(A, i):
    U1 = U(A[4*i  ][0],A[4*i  ][1],A[4*i  ][2])
    U2 = U(A[4*i+1][0],A[4*i+1][1],A[4*i+1][2])
    U3 = U(A[4*i+2][0],A[4*i+2][1],A[4*i+2][2])
    U4 = U(A[4*i+3][0],A[4*i+3][1],A[4*i+3][2])
    return np.kron(np.kron(U1, U2), np.kron(U3, U4))

#Like `U_4squ` - But, optionally, skip the single unitary labelled j.
def U_4squ_skip(A, i, j):
    U1 = np.eye(2) if 4*i   == j else U(A[4*i  ][0],A[4*i  ][1],A[4*i  ][2])
    U2 = np.eye(2) if 4*i+1 == j else U(A[4*i+1][0],A[4*i+1][1],A[4*i+1][2])
    U3 = np.eye(2) if 4*i+2 == j else U(A[4*i+2][0],A[4*i+2][1],A[4*i+2][2])
    U4 = np.eye(2) if 4*i+3 == j else U(A[4*i+3][0],A[4*i+3][1],A[4*i+3][2])
    return np.kron(np.kron(U1, U2), np.kron(U3, U4))

#Given a 1-qubit unitary Ui on site s (0-3), expand it to a 4-qubit unitary.
def expand_on_site(Ui, s):
    if s == 0:
        return np.kron(Ui, np.eye(8))
    elif s == 1:
        return np.kron(np.eye(2), np.kron(Ui, np.eye(4)))
    elif s == 2:
        return np.kron(np.eye(4), np.kron(Ui, np.eye(2)))
    elif s == 3:
        return np.kron(np.eye(8), Ui)
    
#Like `U_4squ`, but it's only the unitary at the one site j. That is, it's the single-qubit
#unitary, but expanded to 4-qubits.
def U_4squ_only(A, j):
    return expand_on_site(U(A[j][0],A[j][1],A[j][2]), j % 4)
    
#The trace inner product of two matrices. Equal to np.trace(A @ inverse(B)), but faster
def trace_prod_inv(A, B):
    # return np.trace(A @ inverse(B))
    return np.conj(np.vdot(A, B))

In [34]:
#Replaces computing the gradient + gradual adjustment, with just rounding each goal directly to its nearest unitary
#and using that. Note: no "alpha" for learning rate anymore.
def gradient_descent3(layers,U_goal, A, iterations):
    nsu = 4*layers #(number of single qubit unitaries)
    stop_flag=0
    Uerr_array = []

    for i in range(iterations):
        if i%10==0:
            print("iteration",i)
        if stop_flag ==1:
            print("Truncation")
            break

        #Kepp running track of the error: inverse(Uf) * U_goal * inverse(Ub). We'll update this dynamically.
        Ub = U_4squ(A, 0)
        for l in range(1, layers):
            Ub = Ub @ CNOTS[(l-1)%3] @ U_4squ(A, l)
        Ui_err = U_goal @ inverse(Ub)
            
        for l in range(0, layers):            
            for s in [0,1,2,3]:
                j = 4*l+s
                
                # RHS
                Ui_err = Ui_err @ U_4squ_only(A, j)
                
                #Compute the partial trace of Ui, to leave just the `s` qubit part
                if s == 0:
                    Ui2 = np.trace(Ui_err.reshape(2,8, 2,8), axis1=1, axis2=3)
                elif s == 1:
                    Ui2 = Ui_err.reshape(2,2,4, 2,2,4)
                    Ui2 = np.trace(Ui2, axis1=0, axis2=3)
                    Ui2 = np.trace(Ui2, axis1=1, axis2=3)
                elif s == 2:
                    Ui2 = Ui_err.reshape(4,2,2, 4,2,2)
                    Ui2 = np.trace(Ui2, axis1=0, axis2=3)
                    Ui2 = np.trace(Ui2, axis1=1, axis2=3)
                elif s == 3:
                    Ui2 = np.trace(Ui_err.reshape(8,2, 8,2), axis1=0, axis2=2)
                
                #Round goal to nearest unitary
                svd = np.linalg.svd(Ui2)
                unitized = svd[0] @ svd[2]
                unitized /= np.sqrt(np.linalg.det(unitized)) #Convert from U(2) to SU(2)

                # Get Euler angles
                beta = 2 * np.arctan(np.abs(unitized[1][0] / unitized[0][0]))
                phia = np.angle(unitized[0][0])
                phib = np.angle(unitized[1][0])
                a = phib - phia
                c = -(phia + phib)
                
                A[j][0]=beta
                A[j][1]=a
                A[j][2]=c
                
                #computing cost function
                # Uk = U(A0,A1,A2)
                Uk = unitized
                Tr = trace_prod_inv(Ui2, unitized)
                Abs = np.abs(Tr)
                Uerr = 16 - Abs
                Uerr_array.append(Uerr)
                if Uerr < 1e-8:
                    stop_flag=1
                    break
                Ui_err = inverse(expand_on_site(unitized, s)) @ Ui_err
                
            Ui_err = CNOTS[l%3] @ Ui_err @ CNOTS[l%3]
            if stop_flag==1:
                break
    return Uerr_array